In [1]:
from platform import python_version

print(python_version())

3.11.7


In [2]:
import sys
print(sys.version)

3.11.7 (main, Dec 15 2023, 12:09:04) [Clang 14.0.6 ]


In [3]:
# In the final discussion with the business partner, the partner was thrilled with the rank-order ability of the model. 
# The partner would like to classify any observation that would fall in the top 5 bins as an event; 
# for simplicity we will say the cutoff is at the 75th percentile. 

# For the API, please return 
# the predicted outcome (variable name is business_outcome), 
# predicted probability (variable name is phat), 
# and all model inputs; 
# the variables should be returned in alphabetical order in the API return.

In [4]:
import numpy as np
import pandas as pd
import sys
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm

import collections as ccc

# import pickle 

In [5]:
print("python version " + sys.version)
print('numpy version ' + np.__version__)
print('pandas version ' + pd.__version__)
print('sklearn version ' + '0.23.1')
print('statsmodels version ' + '0.9.0')

python version 3.11.7 (main, Dec 15 2023, 12:09:04) [Clang 14.0.6 ]
numpy version 1.26.4
pandas version 2.1.4
sklearn version 0.23.1
statsmodels version 0.9.0


In [6]:
# read data
train = pd.read_csv('exercise_26_train.csv')

In [7]:
"""# get a list of all variables not being used

selected_vars = ['x5', 'x12', 'x31', 'x44', 'x53', 'x56', 'x58', 'x62', 'x81', 'x91', 'y']
discard = [x for x in train.columns.tolist() if x not in selected_vars] 

# drop unused variables from dataframe
train.drop(columns=discard, index=1, inplace=True)"""

"# get a list of all variables not being used\n\nselected_vars = ['x5', 'x12', 'x31', 'x44', 'x53', 'x56', 'x58', 'x62', 'x81', 'x91', 'y']\ndiscard = [x for x in train.columns.tolist() if x not in selected_vars] \n\n# drop unused variables from dataframe\ntrain.drop(columns=discard, index=1, inplace=True)"

In [8]:
train.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,y
0,-1.018506,-4.180869,5.703059,-0.522022,-1.678554,tuesday,0.186170,30.162959,1.200073,0.373124,...,0.041910,0.338892,3.524999,-97.715138,1.444637,2.728553,0.718725,-32.945908,2.555359,1
1,-1.919220,0.451107,4.823385,-2.014569,-1.211901,saturday,0.818817,14.490744,4.130146,0.314697,...,0.385252,0.049265,3.413508,-106.064102,0.283217,2.703819,0.723491,-32.166802,15.347209,0
2,-0.865318,-6.066885,5.193225,-0.749215,-0.967170,thursday,0.783379,35.237738,3.391955,0.862954,...,0.560975,0.580471,3.415830,-92.080705,1.079652,2.558499,0.398261,-32.760533,-102.950870,0
3,1.442629,1.972809,4.015218,1.028498,0.277185,sunday,0.809138,3.217519,3.271071,0.708264,...,0.064134,0.751799,3.112920,-95.445212,1.071278,1.886491,0.225542,-32.868644,-20.483119,1
4,0.103217,2.862795,3.677806,-0.116603,0.392040,thursday,0.605821,9.086870,0.020512,0.512565,...,0.688950,0.490738,3.979148,-103.656620,0.502591,2.836184,0.486655,-32.216762,108.555781,1


# Prep the training data

In [10]:
# data prep for training data

train['x12'] = train['x12'].str.replace('$','')
train['x12'] = train['x12'].str.replace(',','')
train['x12'] = train['x12'].str.replace(')','')
train['x12'] = train['x12'].str.replace('(','-')
train['x12'] = train['x12'].astype(float)
train['x63'] = train['x63'].str.replace('%','')
train['x63'] = train['x63'].astype(float)

# mean imputation
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
train_imputed = pd.DataFrame(imputer.fit_transform(train.drop(columns=['y', 'x5', 'x31', 'x82', 'x81'])), 
                             columns=train.drop(columns=['y', 'x5', 'x31', 'x82', 'x81']).columns)
# standardizing the data
std_scaler = StandardScaler()
train_imputed_std = pd.DataFrame(std_scaler.fit_transform(train_imputed), columns=train_imputed.columns)

# create dummy variables

dumb5 = pd.get_dummies(train['x5'], drop_first=True, prefix='x5', prefix_sep='_', dummy_na=True)
train_imputed_std = pd.concat([train_imputed_std, dumb5], axis=1, sort=False)

dumb31 = pd.get_dummies(train['x31'], drop_first=True, prefix='x31', prefix_sep='_', dummy_na=True)
train_imputed_std = pd.concat([train_imputed_std, dumb31], axis=1, sort=False)

dumb81 = pd.get_dummies(train['x81'], drop_first=True, prefix='x81', prefix_sep='_', dummy_na=True)
train_imputed_std = pd.concat([train_imputed_std, dumb81], axis=1, sort=False)

dumb82 = pd.get_dummies(train['x82'], drop_first=True, prefix='x82', prefix_sep='_', dummy_na=True)
train_imputed_std = pd.concat([train_imputed_std, dumb82], axis=1, sort=False)

train_imputed_std = pd.concat([train_imputed_std, train['y']], axis=1, sort=False)

del dumb5, dumb31, dumb81, dumb82

train_imputed_std.head()

,x0,x1,x2,x3,x4,x6,x7,x8,x9,x10,...,x81_June,x81_March,x81_May,x81_November,x81_October,x81_September,x81_nan,x82_Male,x82_nan,y
0,-1.016587,-1.397979,0.705052,-0.530288,-1.702526,-1.092239,-0.692245,-0.920949,-0.437636,1.673726,...,False,False,False,False,False,False,False,False,False,1
1,-1.914220,0.152535,-0.180528,-2.041463,-1.230308,1.107034,-1.239318,1.145254,-0.641408,0.092886,...,False,False,False,False,False,False,False,True,False,0
2,-0.863923,-2.029307,0.191796,-0.760316,-0.982658,0.983841,-0.515099,0.624703,1.270714,0.333830,...,False,False,True,False,False,False,False,False,False,0
3,1.436129,0.661912,-0.994121,1.039584,0.276538,1.073387,-1.632835,0.539459,0.731211,1.216332,...,False,False,False,False,False,False,False,False,False,1
4,0.101299,0.959827,-1.333798,-0.119809,0.392763,0.366595,-1.427953,-1.752741,0.048684,-1.556771,...,False,False,True,False,False,False,False,True,False,1


In [11]:
# train_imputed_std.isna().sum()

# test the model

In [13]:
# selected variables from data scientist code

variables = ['x5_saturday',
 'x81_July',
 'x81_December',
 'x31_japan',
 'x81_October',
 'x5_sunday',
 'x31_asia',
 'x81_February',
 'x91',
 'x81_May',
 'x5_monday',
 'x81_September',
 'x81_March',
 'x53',
 'x81_November',
 'x44',
 'x81_June',
 'x12',
 'x5_tuesday',
 'x81_August',
 'x81_January',
 'x62',
 'x31_germany',
 'x58',
 'x56']

In [14]:
# test the model with selected variables

logit = sm.Logit(train_imputed_std['y'].astype(float), train_imputed_std[variables].astype(float)).fit() 
logit.summary() # final model info

Optimization terminated successfully.
         Current function value: 0.536451
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                40000
Model:                          Logit   Df Residuals:                    39975
Method:                           MLE   Df Model:                           24
Date:                Sun, 09 Jun 2024   Pseudo R-squ.:                  0.2261
Time:                        16:36:37   Log-Likelihood:                -21458.
converged:                       True   LL-Null:                       -27725.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
x5_saturday      -1.2069      0.037    -32.957      0.000      -1.279      -1.135
x81_July          1.0944      0.047     23.061      0.000       1.001       1.187
x81_December      1.0280      0.047     21.782      0.000       0.935       1.120
x31_japan         0.9244      0.056     16.530      0.000       0.815       1.034
x81_October       0.9097      0.047     19.486      0.000       0.818       1.001
x5_sunday        -0.8576      0.036    -23.929      0.000      -0.928      -0.787
x31_asia         -0.7601      0.030    -25.541      0.000      -0.818      -0.702
x81_February      0.8238      0.047     17.614      0.000       0.732       0.916
x91               0.7665      0.013     60.563      0.000       0.742       0.791
x81_May           0.7916      0.047     16.807      0.000       0.699       0.884
x5_monday        -0.6699      0.035    -18.942      0.000      -0.739      -0.601
x81_September     0.6799      0.047     14.600      0.000       0.589       0.771
x81_March         0.6821      0.047     14.634      0.000       0.591       0.773
x53              -0.6227      0.012    -50.366      0.000      -0.647      -0.599
x81_November      0.5594      0.047     12.009      0.000       0.468       0.651
x44              -0.5054      0.012    -41.495      0.000      -0.529      -0.482
x81_June          0.4433      0.047      9.506      0.000       0.352       0.535
x12              -0.3939      0.012    -32.797      0.000      -0.417      -0.370
x5_tuesday       -0.3729      0.035    -10.512      0.000      -0.442      -0.303
x81_August        0.4285      0.047      9.131      0.000       0.337       0.520
x81_January       0.3442      0.047      7.343      0.000       0.252       0.436
x62              -0.2886      0.012    -23.470      0.000      -0.313      -0.265
x31_germany      -0.1670      0.029     -5.668      0.000      -0.225      -0.109
x58               0.2113      0.012     17.518      0.000       0.188       0.235
x56               0.2003      0.012     16.336      0.000       0.176       0.224
=================================================================================
"""

In [15]:
# checking for accuracy
outcomes_train = pd.DataFrame(logit.predict(train_imputed_std[variables].astype(float))).rename(columns={0:'phat'})
outcomes_train['y'] = train_imputed_std['y'].astype(float)
print('The C-Statistic for train is ',roc_auc_score(outcomes_train['y'], outcomes_train['phat']))

The C-Statistic for train is  0.801148915971024


In [16]:
# calculate bin

# quantile-cut creates bins with roughly equal elements inside, intervals are different widths in order to accomplish this
outcomes_train['bin'] = pd.qcut(outcomes_train['phat'], q=4)
outcomes_train.groupby('bin', observed=True).sum()

,phat,y
bin,,
"(0.0025, 0.257]",1468.298025,1550.0
"(0.257, 0.483]",3691.455516,4036.0
"(0.483, 0.71]",5956.922510,5925.0
"(0.71, 0.995]",8313.637256,8393.0


In [ ]:
# The business partner would like to classify any observation that would fall in the top 5 bins as an event; 
# for simplicity we will say the cutoff is at the 75th percentile. 
# For the API, please return the predicted outcome (variable name is business_outcome), 
# predicted probability (variable name is phat), 
# and all model inputs; the variables should be returned in alphabetical order in the API return.

In [17]:
# (0.71, 0.995] is to be used to classify an event when running test data

# Prep the test data

In [19]:
test = pd.read_csv('exercise_26_test.csv') # data will come into the api in this format

In [20]:
test['x12'].head()

0     $5,547.78
1    $-5,483.24
2     $5,515.72
3    $-4,446.84
4     $6,523.20
Name: x12, dtype: object

In [21]:
# prep the data (is this section needed for train or test???)
test['x12'] = test['x12'].str.replace('$','')
test['x12'] = test['x12'].str.replace(',','')
test['x12'] = test['x12'].str.replace(')','')
test['x12'] = test['x12'].str.replace('(','-')
test['x12'] = test['x12'].astype(float)
test['x63'] = test['x63'].str.replace('%','')
test['x63'] = test['x63'].astype(float)

# mean imputation using imputer created for training data
test_imputed = pd.DataFrame(imputer.fit_transform(test.drop(columns=['x5', 'x31',  'x81' ,'x82'])), 
                             columns=test.drop(columns=['x5', 'x31', 'x81', 'x82']).columns) 

# standardizing the data using scaler created for training data
test_imputed_std = pd.DataFrame(std_scaler.fit_transform(test_imputed), columns=test_imputed.columns)

# create dummy variables

dumb5 = pd.get_dummies(test['x5'], drop_first=True, prefix='x5', prefix_sep='_', dummy_na=True)
test_imputed_std = pd.concat([test_imputed_std, dumb5], axis=1, sort=False)

dumb31 = pd.get_dummies(test['x31'], drop_first=True, prefix='x31', prefix_sep='_', dummy_na=True)
test_imputed_std = pd.concat([test_imputed_std, dumb31], axis=1, sort=False)

dumb81 = pd.get_dummies(test['x81'], drop_first=True, prefix='x81', prefix_sep='_', dummy_na=True)
test_imputed_std = pd.concat([test_imputed_std, dumb81], axis=1, sort=False)

dumb82 = pd.get_dummies(test['x82'], drop_first=True, prefix='x82', prefix_sep='_', dummy_na=True)
test_imputed_std = pd.concat([test_imputed_std, dumb82], axis=1, sort=False)

del dumb5, dumb31, dumb81, dumb82

test_imputed_std.head()

,x0,x1,x2,x3,x4,x6,x7,x8,x9,x10,...,x81_July,x81_June,x81_March,x81_May,x81_November,x81_October,x81_September,x81_nan,x82_Male,x82_nan
0,0.032906,-1.121388,-0.368396,-0.620143,-0.660346,-1.097020,-0.109570,3.832449e-01,-0.473929,-0.843230,...,False,False,False,False,False,True,False,False,False,False
1,-1.054929,-0.111436,0.862953,0.000000,0.124365,1.711792,0.062018,-5.469566e-01,1.202016,-1.389640,...,False,False,False,False,True,False,False,False,True,False
2,2.042788,-1.085952,-0.279470,2.265930,0.562102,-0.034467,1.310767,1.295691e+00,0.047288,-0.643391,...,True,False,False,False,False,False,False,False,False,False
3,-0.076328,0.638963,-0.185574,0.651998,1.981841,-1.014726,0.833484,1.713696e+00,-1.343411,0.649169,...,False,False,False,False,False,True,False,False,True,False
4,-0.371127,-0.619683,0.361159,-0.077351,-0.653908,1.513150,0.805422,-3.117883e-16,-1.081063,-0.397805,...,False,False,False,True,False,False,False,False,True,False


# Get predictions on test data

In [23]:
outcomes_test = pd.DataFrame(logit.predict(test_imputed_std[variables].astype(float))).rename(columns={0:'phat'})
outcomes_test.head()

,phat
0,0.366958
1,0.824429
2,0.134487
3,0.471328
4,0.323100


In [24]:
# the probability must land inside this interval to be considered an event (0.71, 0.995]
# probability must be greater than 0.71 and <= 0.995

outcomes_test['business_outcome'] = outcomes_test.apply(lambda row : 1 if row.phat > 0.71 and row.phat <= 0.995 else 0, axis=1)

In [25]:
all = pd.concat([outcomes_test, test_imputed_std], axis=1)
all.head()

,phat,business_outcome,x0,x1,x2,x3,x4,x6,x7,x8,...,x81_July,x81_June,x81_March,x81_May,x81_November,x81_October,x81_September,x81_nan,x82_Male,x82_nan
0,0.366958,0,0.032906,-1.121388,-0.368396,-0.620143,-0.660346,-1.097020,-0.109570,3.832449e-01,...,False,False,False,False,False,True,False,False,False,False
1,0.824429,1,-1.054929,-0.111436,0.862953,0.000000,0.124365,1.711792,0.062018,-5.469566e-01,...,False,False,False,False,True,False,False,False,True,False
2,0.134487,0,2.042788,-1.085952,-0.279470,2.265930,0.562102,-0.034467,1.310767,1.295691e+00,...,True,False,False,False,False,False,False,False,False,False
3,0.471328,0,-0.076328,0.638963,-0.185574,0.651998,1.981841,-1.014726,0.833484,1.713696e+00,...,False,False,False,False,False,True,False,False,True,False
4,0.323100,0,-0.371127,-0.619683,0.361159,-0.077351,-0.653908,1.513150,0.805422,-3.117883e-16,...,False,False,False,True,False,False,False,False,True,False
